### Load section information from the source language

In [18]:
import pandas as pd
import networkx as nx
import json
from collections import Counter
import gzip,json
import re
targetLang = 'fa'

In [20]:
sectionsTargetAll = []
sectionsTarget = {}
c=0
with gzip.open('../gap/multiLanguageFromDumps/sections-articles_%s.json.gz' % targetLang) as f: #this for the freq hq articles
    for l in f:
        c+=1
        if c%1000000 == 0: print(c)
        tmp = json.loads(l.decode())
        sectionsTarget[tmp[1]] = tmp[2]
        for secName in tmp[2]:
                sectionsTargetAll.append(secName.strip())
sectionsTargetFreq = Counter(sectionsTargetAll)


1000000
2000000


### Select the sections name to be translate

* use "freq" to filter by frequecy
* use acc/total to filter by %

In [21]:
#sectionsEnFreq.most_common(50)
total = sum(sectionsTargetFreq.values())
acc =0
secsToEval = []
for n,(sec,freq) in enumerate(sectionsTargetFreq.most_common()):
    acc+= freq
    secsToEval.append(sec)
    #print(n,acc/total,sec)
    if freq < 100:
        print(n)
        print(acc/total)
        break
    #if acc/total > 0.82: #using sections that cover 80% of total
    #    print(n)
    #    print(freq)
    #    break

251
0.7644290801033017


In [22]:
from googletrans import Translator
from random import random
from time import sleep

def translateSecs(seclist=secsToEval,targetLang='es'):
    print('Wait, translating')
    translator = Translator()
    words = ''
    results = ''
    for n,section in enumerate(secsToEval):
        if len(words) + len(section) < 4900: #max allowed by google
            words += '%d) %s \n' % (n , section)
        else:
            results+= translator.translate(words,dest=targetLang).text +'\n'
            sleep(random() * 5)
            #print(results)
            words = '%d) %s \n' % (n , section)

    results+= translator.translate(words,dest=targetLang).text
    d  = dict(re.findall("(\d+)\) (.+)\n",results))
    output ={}
    for n,word  in enumerate(secsToEval):
        try:
            output[word] = d[str(n)]
        except:
            pass
    print('Finish')
    return output

In [88]:
secsEs = translateSecs(targetLang='es')

Wait, translating
Finish


In [89]:
secsFa = translateSecs(targetLang='fa')

Wait, translating
Finish


In [90]:
secsDe = translateSecs(targetLang='de')

Wait, translating
Finish


In [12]:
secsEnFromDe = translateSecs(targetLang='en')

Wait, translating
Finish


In [23]:
secsEnFromFa = translateSecs(targetLang='en')

Wait, translating
Finish


### Export to Excel

In [26]:
## join 3
#df = pd.DataFrame.from_dict(secsEs,orient='index').join(pd.DataFrame.from_dict(secsDe,orient='index'),lsuffix='_es',rsuffix='_de').join(pd.DataFrame.from_dict(secsFa,orient='index'),rsuffix='_fa')
## join 2
df = pd.DataFrame.from_dict(secsEnFromFa,orient='index')
rank = dict([(x,n) for n,(x,y) in enumerate(sectionsTargetFreq.most_common(len(secsToEval)))])
df  = df.join(pd.DataFrame.from_dict(rank,orient='index'),rsuffix='rank')
df = df.sort_values(["0rank"])

In [27]:
df.to_excel('automaticTranslationEnFromFa.xls')